In [123]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42) 

In [126]:
def get_upstream_gene(gene, weight, grn = grn, important_genes = important_genes):
    upstream_genes = grn.iloc[np.where(grn["target"] == gene)[0]]["regulator"]
    if len(upstream_genes) == 0:
        return -1
    for i in range(len(upstream_genes)):
        if upstream_genes.iloc[i] in important_genes.keys():
            if weight <.001:
                #print("weight too small")
                continue
            else:
                important_genes[upstream_genes.iloc[i]] = important_genes[upstream_genes.iloc[i]] + weight
                #get_upstream_gene(upstream_genes.iloc[i], weight*.5, grn, important_genes)
        elif "HK" in upstream_genes.iloc[i]:
            continue
        else:
            if weight*.5 < .001:
                #print("weight too small")
                continue
            else:
                get_upstream_gene(upstream_genes.iloc[i], weight*.5, grn, important_genes)

def Calc_var_captured(genes = genes, strengths = strengths, important_genes = important_genes, grn = grn):
    weight = 1
    weights = np.abs(strengths) / sum(np.abs(strengths))
    for i in range(len(genes)):
        if genes[i] in important_genes.keys():
            if weights[i] <.001:
                #print("weight too small")
                continue
            else:
                important_genes[genes[i]] = important_genes[genes[i]] + weights[i]
                #get_upstream_gene(genes[i], weights[i]*.5, grn, important_genes)
        else:
            if weights[i]*.5 < .001:
                #print("weight too small")
                continue
            else:
                get_upstream_gene(genes[i], weights[i]*.5, grn, important_genes)
        #print(genes[i], "\n", important_genes, "\n")
    print(sum(important_genes.values()))
    print(important_genes)
    
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [125]:
test_data = pd.read_csv("test_data.csv")
test_data = test_data.drop("Unnamed: 0", axis = 1)
train_data = pd.read_csv("train_data.csv")
train_data = train_data.drop("Unnamed: 0", axis = 1)

test_data_X = test_data.drop("time", axis = 1)
test_data_y = test_data["time"]
train_data_X = train_data.drop("time", axis = 1)
train_data_y = train_data["time"]

In [73]:
class DynGenData():
    def __init__(self, mode):
        if mode == "train":
            self.x = torch.from_numpy(train_data_X.to_numpy())
            self.y = torch.from_numpy(train_data_y.to_numpy())
            self.n_samples = train_data_y.to_numpy().shape[0]
        elif mode == "test":
            self.x = torch.from_numpy(test_data_X.to_numpy())
            self.y = torch.from_numpy(test_data_y.to_numpy())
            self.n_samples = test_data_y.to_numpy().shape[0]
            
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

class Net(nn.Module):

    def __init__(self, hidden_size_1, hidden_size_2 = 50):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(train_data_X.shape[1], hidden_size_1)  # 5*5 from image dimension
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.fc3 = nn.Linear(hidden_size_2, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # x = self.fc2(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
train_dataset = DynGenData("train")
test_dataset = DynGenData("test")

batch_size = 64

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size)

criterion = nn.MSELoss()

In [85]:
learning_rates = [.0001] 
hidden_sizes = [750]
l1_lambdas = [.0001]
l2_lambdas = [0]
num_epochs = 300
input_size = train_data_X.shape[1]

for l1_lambda in l1_lambdas:
    for l2_lambda in l2_lambdas:
        for hidden_size in hidden_sizes:
            for learning_rate in learning_rates:
                print("learning rate is... ", learning_rate, " hidden size is... ", hidden_size, "lambda 1 is... ", l1_lambda, "lambda 2 is... ", l2_lambda)
                total_step = len(train_loader)
                model = Net(hidden_size).to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)
                for epoch in range(num_epochs):
                    model.train()
                    for i, (images, labels) in enumerate(train_loader):  
                        images = images.to(device, dtype=torch.float)
                        labels = labels.to(device, dtype=torch.float)
                        outputs = model(images)
                        loss = criterion(outputs, labels)

                        l1_norm = sum(torch.linalg.norm(p, 1) for p in model.parameters())
                        loss = loss + l1_lambda * l1_norm

                        optimizer.zero_grad() 
                        loss.backward()
                        optimizer.step()
                    if epoch % 10 == 0:
                        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

                with torch.no_grad(): # In test phase, we don't need to compute gradients (for memory efficiency)
                    model.eval()
                    mse = []
                    for images, labels in test_loader:
                        images = images.to(device, dtype=torch.float)
                        labels = labels.to(device, dtype=torch.float)
                        outputs = model(images)
                        mse.append(criterion(outputs, labels).item())
                print(np.mean(mse), "\n")

learning rate is...  0.0001  hidden size is...  750 lambda 1 is...  0.0001 lambda 2 is...  0
Epoch [1/300], Step [22/22], Loss: 52131.5312
Epoch [11/300], Step [22/22], Loss: 30922.5000
Epoch [21/300], Step [22/22], Loss: 28066.8730
Epoch [31/300], Step [22/22], Loss: 24512.5527
Epoch [41/300], Step [22/22], Loss: 21881.3496
Epoch [51/300], Step [22/22], Loss: 21080.1543
Epoch [61/300], Step [22/22], Loss: 20937.0215
Epoch [71/300], Step [22/22], Loss: 20878.5352
Epoch [81/300], Step [22/22], Loss: 20840.0840
Epoch [91/300], Step [22/22], Loss: 20813.0723
Epoch [101/300], Step [22/22], Loss: 20793.9219
Epoch [111/300], Step [22/22], Loss: 20780.1758
Epoch [121/300], Step [22/22], Loss: 20770.1406
Epoch [131/300], Step [22/22], Loss: 20763.2520
Epoch [141/300], Step [22/22], Loss: 20758.4941
Epoch [151/300], Step [22/22], Loss: 20755.4375
Epoch [161/300], Step [22/22], Loss: 20753.5566
Epoch [171/300], Step [22/22], Loss: 20752.4414
Epoch [181/300], Step [22/22], Loss: 20752.3496
Epoch 

## Calculate Neural Network variable importances

In [ ]:
model.eval()
# Get baseline model performance
df_tensor = df_to_tensor(test_data_X)
true_vals = df_to_tensor(test_data_y)
baseline = torch.mean((model(df_tensor) - true_vals)**2).item()

differences = []
for i in test_data_X.columns:
    temp = test_data_X.copy()
    temp[i] = np.random.permutation(temp[i])
    df_tensor = df_to_tensor(temp)
    difference = baseline - torch.mean((model(df_tensor) - true_vals)**2).item()
    differences.append(difference)
differences

In [121]:
genes = test_data_X.columns[np.array(differences).argsort()[:25]]
strengths = np.array(differences)[np.array(differences).argsort()[:25]]
print(genes)
print(strengths)

Index(['Target193', 'HK1998', 'Target672', 'HK2076', 'Target709', 'Target860',
       'Target629', 'Target742', 'HK2104', 'Target823', 'HK3018', 'HK3160',
       'HK836', 'HK552', 'HK2261', 'Target654', 'HK2375', 'HK939', 'HK1443',
       'HK2752', 'HK3025', 'HK1711', 'HK2638', 'HK214', 'Target759'],
      dtype='object')
[-14.06054688 -13.19335938 -12.953125   -11.35546875  -9.74414062
  -9.50195312  -9.49804688  -9.203125    -8.88671875  -8.79101562
  -8.4140625   -8.33984375  -8.28125     -8.26757812  -8.1328125
  -7.92773438  -7.67382812  -7.62304688  -7.61914062  -7.59179688
  -7.4140625   -7.32617188  -7.265625    -7.23828125  -7.16992188]


# Compare Important Variables with Driving Genes
The gene names and strengths for the Lasso and Random Forest were entered in manually by copying output from R.
## Neural Network

In [127]:
genes = test_data_X.columns[np.array(differences).argsort()[:25]]
strengths = np.array(differences)[np.array(differences).argsort()[:25]]

important_genes = {"A1_TF1":0, "A2_TF1":0, "A3_TF1":0, "A4_TF1":0, 
                   "B1_TF1":0, "B2_TF1":0, "B3_TF1":0, "B4_TF1":0, "B5_TF1":0, "B6_TF1":0, "B7_TF1":0, "B8_TF1":0, "B9_TF1":0, "B10_TF1":0, "B11_TF1":0, "B12_TF1":0, "B13_TF1":0, "B14_TF1":0,
                   "Burn1_TF1":0, "Burn2_TF1":0, "Burn3_TF1":0, "Burn4_TF1":0,
                   "C1_TF1":0, "C2_TF1":0, "C3_TF1":0, "C4_TF1":0,
                   "D1_TF1":0, "D2_TF1":0, "D3_TF1":0, "D4_TF1":0, "D5_TF1":0}

Calc_var_captured(genes = genes, strengths = strengths, important_genes = important_genes, grn = grn)

0.27018913108077397
{'A1_TF1': 0, 'A2_TF1': 0, 'A3_TF1': 0, 'A4_TF1': 0, 'B1_TF1': 0.046719047702284605, 'B2_TF1': 0.047588666118967296, 'B3_TF1': 0.050241220786939114, 'B4_TF1': 0, 'B5_TF1': 0, 'B6_TF1': 0, 'B7_TF1': 0, 'B8_TF1': 0, 'B9_TF1': 0, 'B10_TF1': 0, 'B11_TF1': 0.021251026936321207, 'B12_TF1': 0, 'B13_TF1': 0, 'B14_TF1': 0, 'Burn1_TF1': 0, 'Burn2_TF1': 0, 'Burn3_TF1': 0, 'Burn4_TF1': 0, 'C1_TF1': 0.01966910800748134, 'C2_TF1': 0, 'C3_TF1': 0.05326085056547047, 'C4_TF1': 0, 'D1_TF1': 0, 'D2_TF1': 0, 'D3_TF1': 0.03145921096330997, 'D4_TF1': 0, 'D5_TF1': 0}


## Lasso

In [128]:
grn = pd.read_csv("data_raw/b3_cellwise_grn.csv")
important_genes = {"A1_TF1":0, "A2_TF1":0, "A3_TF1":0, "A4_TF1":0, 
                   "B1_TF1":0, "B2_TF1":0, "B3_TF1":0, "B4_TF1":0, "B5_TF1":0, "B6_TF1":0, "B7_TF1":0, "B8_TF1":0, "B9_TF1":0, "B10_TF1":0, "B11_TF1":0, "B12_TF1":0, "B13_TF1":0, "B14_TF1":0,
                   "Burn1_TF1":0, "Burn2_TF1":0, "Burn3_TF1":0, "Burn4_TF1":0,
                   "C1_TF1":0, "C2_TF1":0, "C3_TF1":0, "C4_TF1":0,
                   "D1_TF1":0, "D2_TF1":0, "D3_TF1":0, "D4_TF1":0, "D5_TF1":0}
genes = ["HK2477", "Target18", "Target473", "Target892", "Target325", "Target115", "Target918", "Target184", "Target440", "Target860", "Target241", "B13_TF1", "HK1464", "Target468", "Target923", 
         "Target875", "Target842", "HK3883", "Target442", "Target450", "Target574", "Target672", "Target102", "Target39", "Target715"]
strengths = [97.35434, 47.04507, 45.25426, 33.84109, 32.17078, 31.46609, 30.71390, 30.40129, 30.07492, 29.73931, 28.88838, 27.57796, 26.82258, 26.50961, 25.57203, 25.32590, 24.98265, 24.26942, 
             22.60229, 22.28934, 22.18656, -21.01602, 20.89633, 19.93176, 19.67148]

Calc_var_captured(genes = genes, strengths = strengths, important_genes = important_genes, grn = grn)

0.49563377859444796
{'A1_TF1': 0, 'A2_TF1': 0, 'A3_TF1': 0, 'A4_TF1': 0, 'B1_TF1': 0.013707231859771652, 'B2_TF1': 0.013000047377825211, 'B3_TF1': 0.14177612657476477, 'B4_TF1': 0, 'B5_TF1': 0, 'B6_TF1': 0.11739272835955214, 'B7_TF1': 0, 'B8_TF1': 0.01283028553383851, 'B9_TF1': 0, 'B10_TF1': 0, 'B11_TF1': 0, 'B12_TF1': 0, 'B13_TF1': 0.035974222706250586, 'B14_TF1': 0, 'Burn1_TF1': 0, 'Burn2_TF1': 0, 'Burn3_TF1': 0, 'Burn4_TF1': 0, 'C1_TF1': 0, 'C2_TF1': 0, 'C3_TF1': 0, 'C4_TF1': 0.060200186182330306, 'D1_TF1': 0.02747073793154258, 'D2_TF1': 0, 'D3_TF1': 0.07328221206857218, 'D4_TF1': 0, 'D5_TF1': 0}


## Random Forest

In [129]:
genes = ["Target39", "Target716", "Target375", "Target714", "Target715", "Target467", "Target351", "Target892", "Target468", "Target79", "Target34", "Target75", "Target102", "Target18", "Target310", 
         "Target107", "Target746", "Target360", "Target877", "Target741", "Target115", "Target747", "Target717", "Target106", "Target654"]
strengths = [3462669.8, 3344881.9, 2174365.2, 1981037.1, 1295888.4, 1060049.7, 1031058.0, 827125.3, 776734.8, 655976.6, 607313.8, 558519.3, 522553.2, 509298.8, 472516.3, 467724.3, 433526.0, 355922.9, 
             349477.5, 323801.4, 286571.8, 283175.5, 253112.5, 251063.2, 245893.5]
important_genes = {"A1_TF1":0, "A2_TF1":0, "A3_TF1":0, "A4_TF1":0, 
                   "B1_TF1":0, "B2_TF1":0, "B3_TF1":0, "B4_TF1":0, "B5_TF1":0, "B6_TF1":0, "B7_TF1":0, "B8_TF1":0, "B9_TF1":0, "B10_TF1":0, "B11_TF1":0, "B12_TF1":0, "B13_TF1":0, "B14_TF1":0,
                   "Burn1_TF1":0, "Burn2_TF1":0, "Burn3_TF1":0, "Burn4_TF1":0,
                   "C1_TF1":0, "C2_TF1":0, "C3_TF1":0, "C4_TF1":0,
                   "D1_TF1":0, "D2_TF1":0, "D3_TF1":0, "D4_TF1":0, "D5_TF1":0}

Calc_var_captured(genes = genes, strengths = strengths, important_genes = important_genes, grn = grn)

0.7032289163699191
{'A1_TF1': 0, 'A2_TF1': 0, 'A3_TF1': 0, 'A4_TF1': 0, 'B1_TF1': 0.005456961768851211, 'B2_TF1': 0.08833661851559546, 'B3_TF1': 0.0894465881099056, 'B4_TF1': 0, 'B5_TF1': 0, 'B6_TF1': 0.07298256138829275, 'B7_TF1': 0, 'B8_TF1': 0.15257082866450064, 'B9_TF1': 0.009620973339283023, 'B10_TF1': 0, 'B11_TF1': 0, 'B12_TF1': 0.007185923420100566, 'B13_TF1': 0, 'B14_TF1': 0, 'Burn1_TF1': 0, 'Burn2_TF1': 0, 'Burn3_TF1': 0, 'Burn4_TF1': 0, 'C1_TF1': 0.04825433680809178, 'C2_TF1': 0, 'C3_TF1': 0, 'C4_TF1': 0.03407733905633956, 'D1_TF1': 0.08860739328102112, 'D2_TF1': 0, 'D3_TF1': 0.10668939201793744, 'D4_TF1': 0, 'D5_TF1': 0}
